In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [2]:
from evaluator.tester import Tester, train_ds, val_ds, test_ds

e:\RUSHIL\Courses\Agentic_AI\AI_Engineer_Core\ai-pricer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
import re
from common.constants import (
    OPENROUTER_BASE_URL,
    OPENROUTER_API_KEY,
)
from models.item import Item
from openai import OpenAI

SYSTEM_PROMPT = """
You are a price estimator who estimates the price of a product based on its description.
Respond with only the price number in dollars. Do not include currency symbol, just respond with the number.
"""
MODEL = "deepseek/deepseek-r1-0528:free"


def get_messages(item: Item) -> list:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": "Estimate the price of this product" + "\n" + item.summary,
        },
    ]
    return messages


def llm_model(item: Item) -> float:
    openai_client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)
    response = openai_client.chat.completions.create(
        model=MODEL,
        messages=get_messages(item),
    )
    estimated_price = response.choices[0].message.content
    estimated_price = re.sub(r"[^0-9.]", "", estimated_price)
    try:
        estimated_price = float(estimated_price)
    except Exception as err:
        print(f"Error: {err}")
        return 0
    return estimated_price

In [ ]:
Tester.test(llm_model, max_datapoints=10)  # Limiting datapoints due to rate limits